In [73]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
df = pd.read_csv("train.csv")

In [4]:
df

,Id,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,TARGET
0,57094,1.0,3,0,2016-05-19 18:06:27,A FABULOUS 3BR IN MIDTOWN WEST! PERFECT APAR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...,"['Laundry In Unit', 'No Fee', 'Elevator']",40.7647,7039994,-73.9918,4bdc3d8c1aaa90d997ce2cb77680679b,['https://photos.renthop.com/2/7039994_07be01b...,4495,W 50 & AVE 10,medium
1,33389,1.0,1,9225efdfb57a50bf3ec17ebab082f94a,2016-06-16 02:01:49,Renovated Kitchen and Bathroom!,55 River Drive South,"['Dogs Allowed', 'Cats Allowed', 'No Fee']",40.7275,7166774,-74.0322,e5808a5e6cc13988fe596704428d38d5,['https://photos.renthop.com/2/7166774_03cf63a...,2570,55 River Drive South,medium
2,60458,1.0,0,320de7d3cc88e50a7fbbcfde1e825d21,2016-05-04 02:42:50,RARE AND BEST DEAL ON THE MARKET!!!! PERFECT S...,W 77 Street,"['Elevator', 'Hardwood Floors']",40.7798,6962716,-73.9751,d69d4e111612dd12ef864031c1148543,['https://photos.renthop.com/2/6962716_ec7f56f...,1795,22 W 77 Street,low
3,53048,1.0,2,ce6d18bf3238e668b2bf23f4110b7b67,2016-05-12 05:57:56,Newly renovated flex 2 apartment offers the ne...,John Street,"['Swimming Pool', 'Doorman', 'Elevator', 'Fitn...",40.7081,7002458,-74.0065,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7002458_93f4010...,3400,100 John Street,low
4,592,1.0,3,fee4d465932160318364d9d48d272879,2016-06-16 06:06:15,LOW FEE apartments do not come around like thi...,West 16th Street,"['Laundry in Building', 'Laundry in Unit', 'Di...",40.7416,7170465,-74.0025,6fba9b3a8327c607b8b043716efee684,['https://photos.renthop.com/2/7170465_9c3f173...,5695,321 West 16th Street,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,25582,1.0,1,14fdc4b01ae44b025f6c4d28c9097e5f,2016-06-16 02:12:57,Newly renovated bedroom apartment located off ...,29th St,['No pets'],40.7653,7166867,-73.9248,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/7166867_625fde3...,1950,30-95 29th St,low
34542,50013,1.0,0,9b6cf886379a2511f8c633c84028efe7,2016-05-10 03:17:32,All apartments are newly renovated featuring: ...,E 1st St,"['Pre-War', 'Laundry in Unit', 'Dishwasher', '...",40.7239,6989251,-73.9901,5599e962719af3ccc2976855c2d5893c,['https://photos.renthop.com/2/6989251_42a4884...,2600,39 E 1st St,medium
34543,111475,2.0,2,0,2016-04-21 03:29:35,"<![CDATA[2 bedrooms, 5110, Astoria / Long Isla...",50th Avenue,[],40.7437,6905568,-73.9585,e6472c7237327dd3903b3d6f6a94515a,['https://photos.renthop.com/2/6905568_01602e3...,5110,2-01 50th Avenue,low
34544,71184,1.0,2,8754cae39f6e053974aa2337017eb3c1,2016-05-14 02:27:47,CooperCooper.com :: Listing ID #10_0385; 400 W...,400 West 56th Street,"['Laundry in Unit', 'Dishwasher', 'Cats Allowe...",40.7673,7010187,-73.9866,6e5c10246156ae5bdcd9b487ca99d96a,['https://photos.renthop.com/2/7010187_ab1db5d...,3400,400 West 56th Street,medium


In [5]:
df.nunique()

Id                 34546
bathrooms             12
bedrooms               8
building_id         6378
created            34186
description        27757
display_address     6998
features            8245
latitude            2182
listing_id         34546
longitude           1566
manager_id          3134
photos             31992
price               2407
street_address     12398
TARGET                 3
dtype: int64

In [6]:
df.isna().sum()

Id                    0
bathrooms             0
bedrooms              0
building_id           0
created               0
description        1037
display_address      88
features              0
latitude              0
listing_id            0
longitude             0
manager_id            0
photos                0
price                 0
street_address        4
TARGET                0
dtype: int64

In [7]:
df.fillna("", inplace=True)

In [8]:
df[df['building_id'] == '0'].shape[0]

5713

In [9]:
df['building_id'] = df['building_id'].astype('category').cat.rename_categories([i for i in range(df['building_id'].nunique())]).astype('int')
df['manager_id'] = df['manager_id'].astype('category').cat.rename_categories([i for i in range(df['manager_id'].nunique())]).astype('int')

In [10]:
df['created_month'] = pd.to_datetime(df.created).apply(lambda x: x.month)
df['created_day'] = pd.to_datetime(df.created).apply(lambda x: x.day)
df['created_hour'] = pd.to_datetime(df.created).apply(lambda x: x.hour)
df.drop('created', axis=1, inplace=True)

In [11]:
import ast

df['photos'] = df['photos'].apply(lambda x: len(ast.literal_eval(x)))


In [12]:
df['photos']

0         3
1        13
2         6
3         4
4         3
         ..
34541     4
34542     6
34543    15
34544     3
34545     4
Name: photos, Length: 34546, dtype: int64

In [13]:
import re
import seaborn as sns

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from functools import lru_cache



def tokenize(text: str, regex="[A-Za-z0-9]+") -> list: # будем токенизировать с помощью регулярки
    regex = re.compile(regex)
    tokens = regex.findall(text.lower())

    return tokens

def remove_stopwords(
    lemmas, stopwords=stopwords.words("english")):
    """Returns list of lemmas without stopwords"""
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

@lru_cache(maxsize=128) # ускорим лемматизацию кэшированием 
def lemmatize_word(token):
    """Returns lemma"""
    stemmer = PorterStemmer()
    return stemmer.stem(token)

def lemmatize_text(text):
    """Returns list of lemmas"""
    return [lemmatize_word(w) for w in text]

def clean_text(text):
    """Returns list of lemmas without stopwords"""
    tokens = tokenize("".join(text))
    lemmas = lemmatize_text(tokens)

    return remove_stopwords(lemmas)

In [14]:
from multiprocess import Pool # распараллелим процессы

def get_clean_text(df):
    with Pool(8) as p:
        review_lemmas = list(p.imap(clean_text, df))
    return review_lemmas

In [15]:
df['features'].apply(lambda x: ast.literal_eval(x)).explode().nunique()

1247

In [16]:
df['features'] = df['features'].apply(lambda x: " ".join(ast.literal_eval(x)))

In [17]:
df['features_and_description'] = df['features'] + " " + df['description']
df['address'] = df['display_address'] + " " + df['street_address']
df.drop(['display_address', 'street_address', 'features', 'description', "Id"], axis=1, inplace=True)

In [18]:
df

,bathrooms,bedrooms,building_id,latitude,listing_id,longitude,manager_id,photos,price,TARGET,created_month,created_day,created_hour,features_and_description,address
0,1.0,3,0,40.7647,7039994,-73.9918,930,3,4495,medium,5,19,18,Laundry In Unit No Fee Elevator A FABULOUS 3BR...,HOW AMAZING IS THIS MIDTOWN WEST STEAL!! NO FE...
1,1.0,1,3623,40.7275,7166774,-74.0322,2804,13,2570,medium,6,16,2,Dogs Allowed Cats Allowed No Fee Renovated Kit...,55 River Drive South 55 River Drive South
2,1.0,0,1209,40.7798,6962716,-73.9751,2641,6,1795,low,5,4,2,Elevator Hardwood Floors RARE AND BEST DEAL ON...,W 77 Street 22 W 77 Street
3,1.0,2,5152,40.7081,7002458,-74.0065,2816,4,3400,low,5,12,5,Swimming Pool Doorman Elevator Fitness Center ...,John Street 100 John Street
4,1.0,3,6341,40.7416,7170465,-74.0025,1373,3,5695,low,6,16,6,Laundry in Building Laundry in Unit Dishwasher...,West 16th Street 321 West 16th Street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,1.0,1,504,40.7653,7166867,-73.9248,2816,4,1950,low,6,16,2,No pets Newly renovated bedroom apartment loca...,29th St 30-95 29th St
34542,1.0,0,3876,40.7239,6989251,-73.9901,1051,6,2600,medium,5,10,3,Pre-War Laundry in Unit Dishwasher Hardwood Fl...,E 1st St 39 E 1st St
34543,2.0,2,0,40.7437,6905568,-73.9585,2816,15,5110,low,4,21,3,"<![CDATA[2 bedrooms, 5110, Astoria / Long Isl...",50th Avenue 2-01 50th Avenue
34544,1.0,2,3350,40.7673,7010187,-73.9866,1357,3,3400,medium,5,14,2,Laundry in Unit Dishwasher Cats Allowed Dogs A...,400 West 56th Street 400 West 56th Street


In [19]:
df['features_and_description'] = get_clean_text(df['features_and_description'])
df['address'] = get_clean_text(df['address'])

In [20]:
df['features_and_description'] = df['features_and_description'].apply(lambda x: " ".join(x))
df['address'] = df['address'].apply(lambda x: " ".join(x))

In [21]:
df

,bathrooms,bedrooms,building_id,latitude,listing_id,longitude,manager_id,photos,price,TARGET,created_month,created_day,created_hour,features_and_description,address
0,1.0,3,0,40.7647,7039994,-73.9918,930,3,4495,medium,5,19,18,laundri unit elev fabul midtown west perfect a...,amaz midtown west steal
1,1.0,1,3623,40.7275,7166774,-74.0322,2804,13,2570,medium,6,16,2,allow allow renov kitchen bathroom,river drive south river drive south
2,1.0,0,1209,40.7798,6962716,-73.9751,2641,6,1795,low,5,4,2,elev hardwood floor rare best deal market perf...,street street
3,1.0,2,5152,40.7081,7002458,-74.0065,2816,4,3400,low,5,12,5,swim pool doorman elev center laundri unit dis...,john street john street
4,1.0,3,6341,40.7416,7170465,-74.0025,1373,3,5695,low,6,16,6,laundri build laundri unit dishwash hardwood f...,west 16th street west 16th street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,1.0,1,504,40.7653,7166867,-73.9248,2816,4,1950,low,6,16,2,newli renov bedroom apart locat 30th avenu tra...,29th 29th
34542,1.0,0,3876,40.7239,6989251,-73.9901,1051,6,2600,medium,5,10,3,laundri unit dishwash hardwood floor outdoor s...,
34543,2.0,2,0,40.7437,6905568,-73.9585,2816,15,5110,low,4,21,3,cdata bedroom 5110 astoria long island citi lu...,50th avenu 50th avenu
34544,1.0,2,3350,40.7673,7010187,-73.9866,1357,3,3400,medium,5,14,2,laundri unit dishwash allow allow coopercoop l...,west 56th street west 56th street


In [123]:
print(df.price[df['price'] > 20000])
print(df.latitude[df['latitude'] > 40])
print(df.longitude[df['longitude'] > -70])

Series([], Name: price, dtype: int64)
28005    44.6038
Name: latitude, dtype: float64
Series([], Name: longitude, dtype: float64)


In [ ]:

df.drop(df.price[df['price'] > 20000].index, axis=0, inplace=True)
df.drop(df.latitude[df['latitude'] < 40].index, axis=0, inplace=True)
df.drop(df.longitude[df['longitude'] > -70].index, axis=0, inplace=True)

In [118]:
df.longitude[df['longitude'] > -70]


308      0.0
7234     0.0
10426    0.0
20352    0.0
24090    0.0
27868    0.0
28855    0.0
33373    0.0
Name: longitude, dtype: float64

In [127]:
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'},]])

fig.add_trace(go.Pie(values=df['bathrooms'].value_counts(), labels=df['bathrooms'].unique(),  name="Bathrooms"), 1, 1)
fig.add_trace(go.Pie(values=df['bedrooms'].value_counts(), labels=df['bedrooms'].unique(), name="Bedrooms"), 1, 2)
fig.add_trace(go.Pie(values=df['TARGET'].value_counts(), labels=df['TARGET'].unique(), name="Popularity"), 1, 3)
fig.update_traces(hole=.4)
fig.update_layout(
    title_text="Pie distribution",

    annotations=[dict(text='Bathrooms', x=0.11, y=0.5, font_size=14, showarrow=False),
                 dict(text='Bedrooms', x=0.50, y=0.5, font_size=14, showarrow=False),
                 dict(text='Target', x=0.88, y=0.5, font_size=14, showarrow=False)],
    showlegend=False)
fig.show()

In [126]:
fig = make_subplots(rows=1, cols=3, specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'},]])

fig.add_trace(go.Pie(values=df['bathrooms'].value_counts(), labels=df['bathrooms'].unique(),  name="Bathrooms"), 1, 1)
fig.add_trace(go.Pie(values=df['bedrooms'].value_counts(), labels=df['bedrooms'].unique(), name="Bedrooms"), 1, 2)
fig.add_trace(go.Pie(values=df['TARGET'].value_counts(), labels=df['TARGET'].unique(), name="Popularity"), 1, 3)

ValueError: 
The 'specs' argument to make_subplots must be a 2D list of dictionaries with dimensions (1 x 4).
    Received value of type <class 'list'>: [[{'type': 'domain'}, {'type': 'domain'}, {'type': 'domain'}]]

In [71]:

df.drop(df.price[df['price'] > 20000].index, axis=0, inplace=True)

,bathrooms,bedrooms,building_id,latitude,listing_id,longitude,manager_id,photos,price,TARGET,created_month,created_day,created_hour,features_and_description,address
0,1.0,3,0,40.7647,7039994,-73.9918,930,3,4495,medium,5,19,18,laundri unit elev fabul midtown west perfect a...,amaz midtown west steal
1,1.0,1,3623,40.7275,7166774,-74.0322,2804,13,2570,medium,6,16,2,allow allow renov kitchen bathroom,river drive south river drive south
2,1.0,0,1209,40.7798,6962716,-73.9751,2641,6,1795,low,5,4,2,elev hardwood floor rare best deal market perf...,street street
3,1.0,2,5152,40.7081,7002458,-74.0065,2816,4,3400,low,5,12,5,swim pool doorman elev center laundri unit dis...,john street john street
4,1.0,3,6341,40.7416,7170465,-74.0025,1373,3,5695,low,6,16,6,laundri build laundri unit dishwash hardwood f...,west 16th street west 16th street
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34541,1.0,1,504,40.7653,7166867,-73.9248,2816,4,1950,low,6,16,2,newli renov bedroom apart locat 30th avenu tra...,29th 29th
34542,1.0,0,3876,40.7239,6989251,-73.9901,1051,6,2600,medium,5,10,3,laundri unit dishwash hardwood floor outdoor s...,
34543,2.0,2,0,40.7437,6905568,-73.9585,2816,15,5110,low,4,21,3,cdata bedroom 5110 astoria long island citi lu...,50th avenu 50th avenu
34544,1.0,2,3350,40.7673,7010187,-73.9866,1357,3,3400,medium,5,14,2,laundri unit dishwash allow allow coopercoop l...,west 56th street west 56th street


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
df[['description',	'display_address']][5:10]

KeyError: "None of [Index(['description', 'display_address'], dtype='object')] are in the [columns]"

In [ ]:
tfidf = TfidfVectorizer()

vector = tfidf.fit_transform(df['description'][5:10])

In [ ]:
vector.toarray()

array([[0.09099354, 0.        , 0.        , ..., 0.07341302, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08038553, 0.09963579,
        0.        ],
       [0.        , 0.        , 0.07895891, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.07373886, 0.        , ..., 0.        , 0.        ,
        0.07373886]])